In [67]:
import sys
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Bibliotecas para crear el transformador de clusters
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.cluster import KMeans

 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler


from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import OneHotEncoder

In [68]:
prestamos = pd.read_csv('loan_data.csv')

In [69]:
#hay_nan_original = prestamosOriginal['Loan_Status'].isnull().any()
prestamos["Loan_Status"] = prestamos["Loan_Status"].map({'Y': 1, 'N': 0})
prestamos["Loan_Status"].head()
print(prestamos["Loan_Status"].unique())

[0 1]


In [70]:
prestamos["ingresos_cat"] = pd.cut(prestamos["ApplicantIncome"],
                                 bins = [0, 2600, 3333, 4288, 6000, np.inf],
                                   labels = [1, 2, 3, 4, 5])

In [90]:
#creamos los datasets estratificados
from sklearn.model_selection import train_test_split
strat_train_set, strat_test_set = train_test_split(prestamos, 
                                                   test_size=0.2, 
                                                   stratify=prestamos["ingresos_cat"], 
                                                   random_state=42)
 
print('Tamaño del train_set',strat_train_set.shape)
print('Tamaño del test_set',strat_test_set.shape)

Tamaño del train_set (304, 14)
Tamaño del test_set (77, 14)


In [91]:
# Separar las variables predictivas de la variable objetivo
prestamostrain = strat_train_set.drop("Loan_Status", axis=1)
prestamostrain_labels = strat_train_set["Loan_Status"].copy()
prestamostrain_labels.head()

74     1
44     1
178    1
283    1
293    1
Name: Loan_Status, dtype: int64

In [92]:
#Estos miden la capacidad de pago del solicitante y co-solicitante en relación al préstamo
def income_to_loan(X):
    total_income = X[:, [0]] + X[:, [1]]
    return total_income / (X[:, [2]] + 1e-5)

def income_to_loan_name(transformer, feature_names_in):
    return ["income_to_loan_ratio"]
#Estos sirven para analizar la dependencia del ingreso conjunto
def applicant_to_coapplicant(X):
    return X[:, [0]] / (X[:, [1]] + 1e-5)

def applicant_to_coapplicant_name(transformer, feature_names_in):
    return ["applicant_to_coapplicant_ratio"]
#Estos se usan para ver qué porcentaje del ingreso representa el préstamo
def loan_to_income(X):
    total_income = X[:, [0]] + X[:, [1]]
    return X[:, [2]] / (total_income + 1e-5)

def loan_to_income_name(transformer, feature_names_in):
    return ["loan_to_income_ratio"]
#Estos se usan para ver la carga mensual o anual del solicitante
def loan_to_term(X):
    return X[:, [0]] / (X[:, [1]] + 1e-5)

def loan_to_term_name(transformer, feature_names_in):
    return ["loan_to_term_ratio"]

In [93]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
income_to_loan_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(income_to_loan, feature_names_out=income_to_loan_name),
    StandardScaler()
)

applicant_to_coapplicant_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(applicant_to_coapplicant, feature_names_out=applicant_to_coapplicant_name),
    StandardScaler()
)

loan_to_income_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(loan_to_income, feature_names_out=loan_to_income_name),
    StandardScaler()
)

loan_to_term_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(loan_to_term, feature_names_out=loan_to_term_name),
    StandardScaler()
)
from category_encoders import CountEncoder
# Pipeline para variables categóricas Ordinal
cat_count_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    CountEncoder()
)

# Pipeline para variables numéricas restantes
default_num_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    StandardScaler()
)


In [94]:
preprocessing = ColumnTransformer([
    ("income_to_loan", income_to_loan_pipeline, ["ApplicantIncome", "CoapplicantIncome", "LoanAmount"]),
    ("applicant_to_coapplicant", applicant_to_coapplicant_pipeline, ["ApplicantIncome", "CoapplicantIncome"]),
    ("loan_to_income", loan_to_income_pipeline, ["ApplicantIncome", "CoapplicantIncome", "LoanAmount"]),
    ("loan_to_term", loan_to_term_pipeline, ["LoanAmount", "Loan_Amount_Term"]),
    ("cat", cat_count_pipeline, make_column_selector(dtype_include=object))  # <-- actualizamos esto
], remainder=default_num_pipeline)

In [95]:
# Aplica el preprocesamiento automátizado
X = preprocessing.fit_transform(prestamostrain)
print(type(X))
print(X.shape)

<class 'numpy.ndarray'>
(304, 13)


In [96]:
prestamostrainProc = pd.DataFrame(X,
                               columns=preprocessing.get_feature_names_out(),
                               index=prestamostrain.index)
prestamostrainProc.head(2)

,income_to_loan__income_to_loan_ratio,applicant_to_coapplicant__applicant_to_coapplicant_ratio,loan_to_income__loan_to_income_ratio,loan_to_term__loan_to_term_ratio,cat__0,cat__1,cat__2,cat__3,cat__4,cat__5,cat__6,remainder__Credit_History,remainder__ingresos_cat
74,-0.352710,1.461968,0.356319,0.034177,1.0,237.0,130.0,195.0,223.0,275.0,96.0,0.400381,1.177683
44,-0.232557,-0.825365,0.024008,0.074821,1.0,237.0,174.0,46.0,81.0,275.0,96.0,0.400381,0.367693


## Entranemiento de los modelos

In [97]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix, classification_report

## SVC Maquina de soporte vectorial

In [98]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_predict
svc_cls = make_pipeline(preprocessing, SVC(C=10, gamma='auto', class_weight='balanced'))

y_pred_svc = cross_val_predict(svc_cls, prestamostrain, prestamostrain_labels, cv=3)

y_pred_svc[:10]

array([0, 1, 0, 0, 0, 1, 1, 1, 1, 1], dtype=int64)

In [99]:
precision_svc = precision_score(prestamostrain_labels, y_pred_svc)
recall_svc = recall_score(prestamostrain_labels, y_pred_svc)
accuracy_svc = accuracy_score(prestamostrain_labels, y_pred_svc)

print(f"Precisión: {precision_svc}")
print(f"Recall: {recall_svc}")
print(f"Accuracy: {accuracy_svc}")

print("Matriz de confusión:\n", confusion_matrix(prestamostrain_labels, y_pred_svc))
print("Reporte de clasificación:\n", classification_report(prestamostrain_labels, y_pred_svc))

Precisión: 0.7709251101321586
Recall: 0.8215962441314554
Accuracy: 0.7039473684210527
Matriz de confusión:
 [[ 39  52]
 [ 38 175]]
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.51      0.43      0.46        91
           1       0.77      0.82      0.80       213

    accuracy                           0.70       304
   macro avg       0.64      0.63      0.63       304
weighted avg       0.69      0.70      0.70       304



## Random Forest

In [100]:
from sklearn.ensemble import RandomForestClassifier
 
forest_clf = make_pipeline(
    preprocessing,
    RandomForestClassifier(random_state=42)
)

In [101]:
y_pred_forest = cross_val_predict(forest_clf, prestamostrain, prestamostrain_labels, cv=10)

In [102]:
y_pred_forest

array([1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,

In [103]:
# Random Forest no tiene decision_function, pero si genera la probabilidad de pertenecer a cada clase. 
y_probas_forest = cross_val_predict(forest_clf, prestamostrain, prestamostrain_labels, cv=3,
                                    method="predict_proba")

In [104]:
y_probas_forest[:10]

array([[0.15, 0.85],
       [0.27, 0.73],
       [0.19, 0.81],
       [0.59, 0.41],
       [0.27, 0.73],
       [0.08, 0.92],
       [0.14, 0.86],
       [0.26, 0.74],
       [0.76, 0.24],
       [0.07, 0.93]])

In [105]:
from sklearn.metrics import precision_score, recall_score, accuracy_score
forest_presicion = precision_score(prestamostrain_labels,y_pred_forest, average='macro')

In [106]:
forest_presicion

0.7837818895430333

In [107]:
from sklearn.metrics import precision_score, recall_score, accuracy_score

# Calcular métricas
accuracy = accuracy_score(prestamostrain_labels, y_pred_forest)

print(f"Accuracy: {accuracy}")

Accuracy: 0.7993421052631579


In [108]:
# Entrenar en todo el conjunto de entrenamiento
forest_clf.fit(prestamostrain, prestamostrain_labels)

# Predecir en el conjunto de prueba
prestamostest = strat_test_set.drop("Loan_Status", axis=1)
prestamostest_labels = strat_test_set["Loan_Status"].copy()

y_pred_test = forest_clf.predict(prestamostest)

# Evaluar en el conjunto de prueba
test_precision = precision_score(prestamostest_labels, y_pred_test, average='macro')
print(f"Precisión en test: {test_precision}")

Precisión en test: 0.8465160075329567


In [109]:
print("Accuracy:", accuracy_score(prestamostrain_labels, y_pred_forest))

Accuracy: 0.7993421052631579


## LogisticRegression

In [110]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

# Usar regresión logística en lugar de lineal
log_reg = make_pipeline(preprocessing, LogisticRegression(max_iter=2000))


# Predicciones de clase
y_pred = cross_val_predict(log_reg, prestamostrain, prestamostrain_labels, cv=3)
print("Predicciones de clase:", y_pred[:5])

# Predicciones de probabilidad
y_proba = cross_val_predict(log_reg, prestamostrain, prestamostrain_labels, cv=3, method='predict_proba')[:, 1]
print("Probabilidades:", y_proba[:5])

Predicciones de clase: [1 1 1 0 1]
Probabilidades: [0.76596558 0.74870967 0.80463349 0.49201319 0.67432929]


In [111]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print("Accuracy:", accuracy_score(prestamostrain_labels, y_pred))
print("Matriz de confusión:\n", confusion_matrix(prestamostrain_labels, y_pred))
print("Reporte de clasificación:\n", classification_report(prestamostrain_labels, y_pred))

Accuracy: 0.8223684210526315
Matriz de confusión:
 [[ 46  45]
 [  9 204]]
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.84      0.51      0.63        91
           1       0.82      0.96      0.88       213

    accuracy                           0.82       304
   macro avg       0.83      0.73      0.76       304
weighted avg       0.82      0.82      0.81       304



In [112]:
df = prestamostrain.copy()
df["Loan_Status"] = prestamostrain_labels

correlaciones = df.corr(numeric_only=True)["Loan_Status"].sort_values(ascending=False)
print(correlaciones)

Loan_Status          1.000000
Credit_History       0.581380
LoanAmount           0.017546
CoapplicantIncome    0.013778
ApplicantIncome      0.012895
Loan_Amount_Term    -0.073363
Name: Loan_Status, dtype: float64


## Afinamiento de datos

## GridSearchCV

In [115]:
from sklearn.model_selection import GridSearchCV

In [116]:
param_grid = {
    "randomforestclassifier__n_estimators": [200, 300],
    "randomforestclassifier__max_depth": [5, 10, 15],  
    "randomforestclassifier__max_features": ["sqrt", "log2"],  
    "randomforestclassifier__min_samples_split": [5, 10], 
    "randomforestclassifier__min_samples_leaf": [2, 4],  
    "randomforestclassifier__class_weight": ["balanced"] 
}

grid_search = GridSearchCV(forest_clf, param_grid, cv=15)

In [117]:
import time
 
inicio = time.time()
grid_search.fit(prestamostrain,  prestamostrain_labels) 
fin = time.time()
(fin - inicio) / 60 

1.925028963883718

In [118]:
cv_res = pd.DataFrame(grid_search.cv_results_)

In [119]:
cv_res[:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_randomforestclassifier__class_weight,param_randomforestclassifier__max_depth,param_randomforestclassifier__max_features,param_randomforestclassifier__min_samples_leaf,param_randomforestclassifier__min_samples_split,param_randomforestclassifier__n_estimators,...,split8_test_score,split9_test_score,split10_test_score,split11_test_score,split12_test_score,split13_test_score,split14_test_score,mean_test_score,std_test_score,rank_test_score
0,0.119381,0.001591,0.009076,0.000389,balanced,5,sqrt,2,5,200,...,0.8,0.85,0.80,0.9,0.8,0.80,0.9,0.800476,0.073675,7
1,0.174077,0.014808,0.010417,0.000422,balanced,5,sqrt,2,5,300,...,0.8,0.85,0.80,0.9,0.8,0.80,0.9,0.800476,0.073675,7
2,0.123974,0.006294,0.010244,0.002610,balanced,5,sqrt,2,10,200,...,0.8,0.85,0.85,0.9,0.8,0.80,0.9,0.803968,0.080450,1
3,0.169766,0.002120,0.010274,0.000442,balanced,5,sqrt,2,10,300,...,0.8,0.85,0.85,0.9,0.8,0.80,0.9,0.803968,0.080450,1
4,0.121032,0.002996,0.009125,0.000740,balanced,5,sqrt,4,5,200,...,0.8,0.85,0.80,0.9,0.8,0.75,0.9,0.793968,0.074540,15


In [120]:
#mejores parametros
print(grid_search.best_params_)
print(grid_search.best_score_)

{'randomforestclassifier__class_weight': 'balanced', 'randomforestclassifier__max_depth': 5, 'randomforestclassifier__max_features': 'sqrt', 'randomforestclassifier__min_samples_leaf': 2, 'randomforestclassifier__min_samples_split': 10, 'randomforestclassifier__n_estimators': 200}
0.8039682539682541


# Evaluar el modelo con el "test set"

In [121]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.utils import resample
import numpy as np


final_model = grid_search.best_estimator_


X_test = strat_test_set.drop("Loan_Status", axis=1)
y_test = strat_test_set["Loan_Status"]


final_predictions = final_model.predict(X_test)
final_proba = final_model.predict_proba(X_test)[:, 1]


accuracy = accuracy_score(y_test, final_predictions)
print(" Resultados en Test Set:")
print(f"Accuracy     : {accuracy:.4f}")

 Resultados en Test Set:
Accuracy     : 0.8961


In [122]:
n_bootstraps = 1000
logloss_bootstrap = []

for _ in range(n_bootstraps):
    X_resampled, y_resampled = resample(X_test, y_test)
    
    probs_resampled = final_model.predict_proba(X_resampled)[:, 1]
    
    logloss_resampled = log_loss(y_resampled, probs_resampled)
    logloss_bootstrap.append(logloss_resampled)

lower = np.percentile(logloss_bootstrap, 2.5)
upper = np.percentile(logloss_bootstrap, 97.5)

print(f"\nIC 95% del Log Loss (bootstrap): [{lower:.4f}, {upper:.4f}]")


IC 95% del Log Loss (bootstrap): [0.3911, 0.5104]
